**Заголовок** Определение стоимости автомобилей

**Описание проекта:**<br>Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Задача: построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

**Цель исследования:** Подготовить данные для предсказания. Обучить модели, определить наилучшую. Сделать предсказания на лучшей модели.

**Ход исследования:**
- Подготовка данных: загрузка и изучение общей информации из представленного датасета.
- Предобработка данных: преобразование данных в нужные типы, работа с пропусками и дубликатами, добавление нужных столбцов. Работа с выбросами и аномалиями.
- Обучение двух моделей: Линейной регресии и модели градиентного бустинга, а если быть точнее - LightGBM
- Определение лучшей модели и предсказание на тестовой выборке.

**Общий вывод:** резюмирование полученных результатов, формулировка ключевых выводов и рекомендаций.

In [1]:
%pip install scikit-learn -q -U 
%pip install lightgbm -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import warnings
import time

from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from sklearn.impute import SimpleImputer

from sklearn.preprocessing import (
    OneHotEncoder,
    MinMaxScaler,
    StandardScaler,
    RobustScaler
)

from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import root_mean_squared_error

from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression

In [3]:
warnings.filterwarnings('ignore')

In [4]:
TEST_SIZE = 0.25
RANDOM_STATE = 42

## Подготовка данных

In [5]:
df_autos = pd.read_csv(r'../datasets/autos.csv')
df_autos.info()
display(df_autos.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  Repaired           283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [6]:
df_autos.DateCrawled = pd.to_datetime(df_autos.DateCrawled, format='%Y-%m-%d %H:%M:%S')
df_autos.DateCreated = pd.to_datetime(df_autos.DateCreated, format='%Y-%m-%d %H:%M:%S')
df_autos.LastSeen = pd.to_datetime(df_autos.LastSeen, format='%Y-%m-%d %H:%M:%S')

In [7]:
df_autos.columns = df_autos.columns.str.replace(r'(?<!^)(?=[A-Z])', '_', regex=True).str.lower()

In [8]:
df_autos.info()
display(df_autos.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   date_crawled        354369 non-null  datetime64[ns]
 1   price               354369 non-null  int64         
 2   vehicle_type        316879 non-null  object        
 3   registration_year   354369 non-null  int64         
 4   gearbox             334536 non-null  object        
 5   power               354369 non-null  int64         
 6   model               334664 non-null  object        
 7   kilometer           354369 non-null  int64         
 8   registration_month  354369 non-null  int64         
 9   fuel_type           321474 non-null  object        
 10  brand               354369 non-null  object        
 11  repaired            283215 non-null  object        
 12  date_created        354369 non-null  datetime64[ns]
 13  number_of_pictures  354369 no

,date_crawled,price,vehicle_type,registration_year,gearbox,power,model,kilometer,registration_month,fuel_type,brand,repaired,date_created,number_of_pictures,postal_code,last_seen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31,0,60437,2016-04-06 10:17:21


In [9]:
df_autos.describe()

,price,registration_year,power,kilometer,registration_month,number_of_pictures,postal_code
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


In [10]:
df_autos = df_autos.drop('number_of_pictures', axis=1)

Избавляемся от признака "количество картинок", так как он неинформативен. Во всех строках значение 0.

In [11]:
df_autos = df_autos[df_autos.price >= 500]
df_autos = df_autos[(df_autos.power >= 20) & (df_autos.power <= 500)]
df_autos = df_autos[(df_autos.registration_year >= 1980) & (df_autos.registration_year <= 2016)]
df_autos = df_autos[df_autos.registration_month.isin(range(1,13))]

In [12]:
max_date_crawled = df_autos.date_crawled.max()
df_autos = df_autos[
    (df_autos.registration_year < max_date_crawled.year) | 
    (
        (df_autos.registration_year == max_date_crawled.year) & 
        (df_autos.registration_month <= max_date_crawled.month)
    )
]

Отбираем данные так, чтобы дата регистрации автомобиля не была больше максимальной даты скачивания анкеты из базы.

In [13]:
df_autos.describe()

,price,registration_year,power,kilometer,registration_month,postal_code
count,260137.000000,260137.000000,260137.000000,260137.000000,260137.000000,260137.000000
mean,5248.962170,2003.314450,123.642692,127537.028566,6.381076,51600.800851
std,4634.393844,5.599016,53.400955,36793.346468,3.352100,25709.387199
min,500.000000,1980.000000,20.000000,5000.000000,1.000000,1067.000000
25%,1600.000000,2000.000000,82.000000,125000.000000,3.000000,31224.000000
50%,3600.000000,2004.000000,116.000000,150000.000000,6.000000,50858.000000
75%,7500.000000,2007.000000,150.000000,150000.000000,9.000000,72336.000000
max,20000.000000,2016.000000,500.000000,150000.000000,12.000000,99998.000000


Избавляемся от выбросов и аномалий.

In [14]:
print('Количество дубликатов: ', df_autos.duplicated().sum())
df_autos = df_autos.drop_duplicates()
print('Количество дубликатов: ', df_autos.duplicated().sum())

Количество дубликатов:  4
Количество дубликатов:  0


In [15]:
pd.DataFrame(round(df_autos.isna().mean()*100,1)).style.background_gradient('coolwarm')

,0
date_crawled,0.000000
price,0.000000
vehicle_type,1.200000
registration_year,0.000000
gearbox,1.200000
power,0.000000
model,2.800000
kilometer,0.000000
registration_month,0.000000
fuel_type,3.100000


In [16]:
df_autos = df_autos.dropna(subset=['model'])
df_autos.vehicle_type = df_autos.groupby(['brand', 'model'])['vehicle_type'].transform(lambda x: x.mode()[0] if not x.mode().empty\
                                                                                     else None)
df_autos = df_autos.fillna('unknown')

In [17]:
pd.DataFrame(round(df_autos.isna().mean()*100,1)).style.background_gradient('coolwarm')

,0
date_crawled,0.000000
price,0.000000
vehicle_type,0.000000
registration_year,0.000000
gearbox,0.000000
power,0.000000
model,0.000000
kilometer,0.000000
registration_month,0.000000
fuel_type,0.000000


Меняем значения в признаке тип автомобиля по моде и так же ставим заглушку для оставшихся NaN значений.

In [18]:
for i in ['date_crawled', 'date_created', 'last_seen']:
    df_autos[f'{i}_year'] = df_autos[i].dt.year
    df_autos[f'{i}_month'] = df_autos[i].dt.month
    df_autos[f'{i}_day'] = df_autos[i].dt.day

In [19]:
df_autos = df_autos.drop(['date_crawled', 'date_created', 'last_seen'], axis=1)

In [20]:
df_autos.fuel_type = df_autos.fuel_type.str.replace('petrol', 'gasoline', regex=True)
df_autos.fuel_type.unique()

array(['gasoline', 'unknown', 'lpg', 'other', 'hybrid', 'cng', 'electric'],
      dtype=object)

In [21]:
print('Количество дубликатов: ', df_autos.duplicated().sum())
df_autos = df_autos.drop_duplicates()
print('Количество дубликатов: ', df_autos.duplicated().sum())

Количество дубликатов:  6117
Количество дубликатов:  0


In [22]:
df_autos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 246849 entries, 2 to 354368
Data columns (total 21 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   price               246849 non-null  int64 
 1   vehicle_type        246849 non-null  object
 2   registration_year   246849 non-null  int64 
 3   gearbox             246849 non-null  object
 4   power               246849 non-null  int64 
 5   model               246849 non-null  object
 6   kilometer           246849 non-null  int64 
 7   registration_month  246849 non-null  int64 
 8   fuel_type           246849 non-null  object
 9   brand               246849 non-null  object
 10  repaired            246849 non-null  object
 11  postal_code         246849 non-null  int64 
 12  date_crawled_year   246849 non-null  int64 
 13  date_crawled_month  246849 non-null  int64 
 14  date_crawled_day    246849 non-null  int64 
 15  date_created_year   246849 non-null  int64 
 16  da

Сами по себе признаки в datetime не информативны для моделей, поэтому мы вычленили из даты нужные нам признаки.

Были загрежены dateframe. Изучены данные. Заполнены пропущенные значения и обработаны аномалии в столбцах. Удалены неинформативнеы признаки и добавлены новые. Удалены дубли и обработаны уникальные значения в признаке тип топлива.

## Обучение и анализ моделей

In [23]:
X_train, X_test, y_train, y_test = train_test_split(df_autos.drop('price', axis=1), 
                                                    df_autos.price, 
                                                    test_size=TEST_SIZE, 
                                                    random_state=RANDOM_STATE)
X_valid, X_test, y_valid, y_test = train_test_split(X_test,
                                                   y_test,
                                                   test_size=TEST_SIZE,
                                                   random_state=RANDOM_STATE)

### LinearRegression

In [24]:
ohe_pipe = Pipeline(
    [
        (
            'simpleImputer_ohe', 
            SimpleImputer(missing_values=np.nan, strategy='most_frequent')
        ),
        (
            'ohe', 
            OneHotEncoder(drop='first', handle_unknown='ignore', sparse_output=False)
        )
    ]
) 

In [25]:
ohe_columns = X_train.select_dtypes(include='object').columns.to_list()
num_columns = X_train.select_dtypes(include='number').columns.to_list()

data_preprocessor = ColumnTransformer(
    [
        ('ohe', ohe_pipe, ohe_columns),
        ('num', MinMaxScaler(), num_columns)
    ],
    remainder='passthrough'
)

pipe_final_linear = Pipeline(
    [
        ('preprocessor', data_preprocessor),
        ('models', LinearRegression())
    ]
)    
    
    
param_grid_linear = {
    'preprocessor__num': [
        MinMaxScaler(), 
        StandardScaler(), 
        RobustScaler(),
    ]
}

In [26]:
%%time
rscv_linear = RandomizedSearchCV(
    pipe_final_linear,
    param_grid_linear,
    scoring='neg_root_mean_squared_error',
    random_state=RANDOM_STATE,
    n_jobs=1,
    error_score='raise',
    verbose=3).fit(X_train, y_train)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV 1/5] END preprocessor__num=MinMaxScaler();, score=-2365.184 total time=   5.9s
[CV 2/5] END preprocessor__num=MinMaxScaler();, score=-2334.574 total time=   5.8s
[CV 3/5] END preprocessor__num=MinMaxScaler();, score=-2367.600 total time=   5.8s
[CV 4/5] END preprocessor__num=MinMaxScaler();, score=-2369.302 total time=   5.8s
[CV 5/5] END preprocessor__num=MinMaxScaler();, score=-2374.904 total time=   5.8s
[CV 1/5] END preprocessor__num=StandardScaler();, score=-2365.184 total time=   5.9s
[CV 2/5] END preprocessor__num=StandardScaler();, score=-2334.574 total time=   5.8s
[CV 3/5] END preprocessor__num=StandardScaler();, score=-2367.600 total time=   5.5s
[CV 4/5] END preprocessor__num=StandardScaler();, score=-2369.302 total time=   5.5s
[CV 5/5] END preprocessor__num=StandardScaler();, score=-2374.904 total time=   5.6s
[CV 1/5] END preprocessor__num=RobustScaler();, score=-2365.184 total time=   5.8s
[CV 2/5] END prep

In [27]:
pipe_linear = pipe_final_linear.set_params(**rscv_linear.best_params_) # type: ignore

start_fit_time_lnr = time.time()
pipe_linear = pipe_linear.fit(X_train, y_train)
end_fit_time_lnr = time.time()
fit_time_linear = end_fit_time_lnr - start_fit_time_lnr

In [28]:
start_linear = time.time()
y_pred_linear = pipe_linear.predict(X_valid)
end_linear = time.time()
pred_time_linear = end_linear - start_linear

In [29]:
rmse_linear = root_mean_squared_error(y_valid, y_pred_linear)

In [30]:
print(f'RMSE на валидационной выборке: {rmse_linear}')
print(f'Время обучения: {fit_time_linear:.2f}s')
print(f'Время предсказания {pred_time_linear:.2f}s')

RMSE на валидационной выборке: 2364.4251338600634
Время обучения: 7.66s
Время предсказания 0.21s


### LGBMRegressor

In [31]:
#Преобразуем категориальные данные в тип category, чтобы модель корректно обработала значения.

for i in [X_train, X_valid, X_test]:
    for col in i.select_dtypes(include='object').columns.to_list():
        i[col] = i[col].astype('category')

In [32]:
display(X_train.info())
display(X_valid.info())
display(X_test.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 185136 entries, 212294 to 173024
Data columns (total 20 columns):
 #   Column              Non-Null Count   Dtype   
---  ------              --------------   -----   
 0   vehicle_type        185136 non-null  category
 1   registration_year   185136 non-null  int64   
 2   gearbox             185136 non-null  category
 3   power               185136 non-null  int64   
 4   model               185136 non-null  category
 5   kilometer           185136 non-null  int64   
 6   registration_month  185136 non-null  int64   
 7   fuel_type           185136 non-null  category
 8   brand               185136 non-null  category
 9   repaired            185136 non-null  category
 10  postal_code         185136 non-null  int64   
 11  date_crawled_year   185136 non-null  int64   
 12  date_crawled_month  185136 non-null  int64   
 13  date_crawled_day    185136 non-null  int64   
 14  date_created_year   185136 non-null  int64   
 15  date_created

None

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46284 entries, 159408 to 57087
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   vehicle_type        46284 non-null  category
 1   registration_year   46284 non-null  int64   
 2   gearbox             46284 non-null  category
 3   power               46284 non-null  int64   
 4   model               46284 non-null  category
 5   kilometer           46284 non-null  int64   
 6   registration_month  46284 non-null  int64   
 7   fuel_type           46284 non-null  category
 8   brand               46284 non-null  category
 9   repaired            46284 non-null  category
 10  postal_code         46284 non-null  int64   
 11  date_crawled_year   46284 non-null  int64   
 12  date_crawled_month  46284 non-null  int64   
 13  date_crawled_day    46284 non-null  int64   
 14  date_created_year   46284 non-null  int64   
 15  date_created_month  46284 non-n

None

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15429 entries, 331455 to 327783
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   vehicle_type        15429 non-null  category
 1   registration_year   15429 non-null  int64   
 2   gearbox             15429 non-null  category
 3   power               15429 non-null  int64   
 4   model               15429 non-null  category
 5   kilometer           15429 non-null  int64   
 6   registration_month  15429 non-null  int64   
 7   fuel_type           15429 non-null  category
 8   brand               15429 non-null  category
 9   repaired            15429 non-null  category
 10  postal_code         15429 non-null  int64   
 11  date_crawled_year   15429 non-null  int64   
 12  date_crawled_month  15429 non-null  int64   
 13  date_crawled_day    15429 non-null  int64   
 14  date_created_year   15429 non-null  int64   
 15  date_created_month  15429 non-

None

In [33]:
pipe_final = Pipeline(
    [
        ('models', LGBMRegressor())
    ]
)    
    
    
param_grid = {
    'models__num_leaves': range(5,101),
    'models__max_depth': range(5,101)
}

In [34]:
%%time
rscv = RandomizedSearchCV(
    pipe_final,
    param_grid,
    scoring='neg_root_mean_squared_error',
    random_state=RANDOM_STATE,
    n_jobs=-1,
    error_score='raise',
    verbose=3).fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006309 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 946
[LightGBM] [Info] Number of data points in the train set: 185136, number of used features: 17
[LightGBM] [Info] Start training from score 5273.962660
CPU times: total: 5.94 s
Wall time: 52.5 s


In [35]:
pipe_lgbmr = pipe_final.set_params(**rscv.best_params_) # type: ignore

start_fit_time_lgbmr = time.time()
pipe_lgbmr = pipe_lgbmr.fit(X_train, y_train)
end_fit_time_lgbmr = time.time()
fit_time_lgbmr = end_fit_time_lgbmr - start_fit_time_lgbmr

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006543 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 946
[LightGBM] [Info] Number of data points in the train set: 185136, number of used features: 17
[LightGBM] [Info] Start training from score 5273.962660


In [36]:
start_lgbmr = time.time()
y_pred = pipe_lgbmr.predict(X_valid)
end_lgbmr = time.time()
pred_time_lgbmr = end_lgbmr - start_lgbmr

In [37]:
rmse_lgbmr = root_mean_squared_error(y_valid, y_pred)

In [38]:
print(f'RMSE на валидационной выборке: {rmse_lgbmr}')
print(f'Время обучения: {fit_time_lgbmr:.2f}s')
print(f'Время предсказания {pred_time_lgbmr:.2f}s')

RMSE на валидационной выборке: 1488.5165560218281
Время обучения: 1.40s
Время предсказания 0.19s


Обе модели уложились в необходимый порог RMSE, но модель LGBMRegressor сделала это лучше всего. По всем показателям, кроме времени предсказания LGBMRegressor выигрывает.

## Проверка лучшей модели

In [39]:
print(f'Параметры лучшей модели: {rscv.best_params_}') # type: ignore

Параметры лучшей модели: {'models__num_leaves': 97, 'models__max_depth': 13}


In [40]:
start_best = time.time()
y_pred_best = rscv.best_estimator_.predict(X_test) # type: ignore
end_best = time.time()
pred_time_best = end_best - start_best
rmse_best_model = root_mean_squared_error(y_test, y_pred_best)

In [41]:
print(f'RMSE на тестовой выборке: {rmse_best_model:.2f}')
print(f'Время предсказания: {pred_time_best:.2f}s')

RMSE на тестовой выборке: 1527.59
Время предсказания: 0.16s


## Вывод

- Были загружены и обработаны данные. Найдены и устранены аномалии с пропусками. Избавились от неинформативных признаков.
- Обучены модели: LinearRegression и LGBMRegressor
- Проанализированы и определена LGBMRegressor как наиболее эффективная с точки зрения запроса бизнеса модель.
- Проверено качество лучшей модели на тестовой выборке: RMSE на тестовой выборке: 1527.59<br>Время предсказания: 0.16s